In [1]:
from bs4 import BeautifulSoup
import requests
from PIL import Image
import requests
from io import BytesIO
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import schedule
import time

%matplotlib inline

In [2]:
def get_relative_results(area, search_item):
    url = 'https://{}.craigslist.org/search/sss?query={}&sort=rel'.format(area.replace(" ", "").lower(), search_item)
    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    search_results = soup.find("div", {"id": "sortable-results"})
    items = search_results.find("ul")
    num_items = items.findAll("li", {"class": "result-row"})
    class_list = []
    
    for row in items:
        if(str(row).strip().startswith("<h4")):
            return class_list
        else:
            class_list.append(row)

def get_urls(class_list):
    urls = set()
    for x in class_list:
        soup = BeautifulSoup(str(x))
        search_results = soup.find_all("a", href=True)
        for url in search_results:
            if(url['href'].startswith("https")):
                urls.add(url['href'])
    return urls

def get_image_urls(url_list):
    imgs = set()
    
    for url in url_list:
        r  = requests.get(url)
        data = r.text
        soup = BeautifulSoup(data)
        img = soup.findAll("img")

        for row in img:
            imgs.add((url, img[0]['src']))
            break
        
    return imgs

def display_imgs(imgs):
    for x in img_urls:
        response = requests.get(x[1])
        img = Image.open(BytesIO(response.content))
        plt.figure()
        plt.imshow(img)
        plt.title(x[0])

def job():
    area = "phoenix"
    search_item = "vance and hines exhaust"
    char = "%20"
    search_item = search_item.replace(" ", char).lower()

    results = get_relative_results(area, search_item)
    urls = get_urls(results)
    img_urls = get_image_urls(urls)
    display_imgs(img_urls)


In [3]:
# display_imgs(img_urls)

# schedule.every(10).minutes.do(job)
# schedule.every().hour.do(job)
schedule.every().day.at("15:30").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 